In [191]:
# libraries
import random
import re


In [192]:
spaceMapping = {
    'A': 0, 'B': 1, 'C': 2,
    'D': 3, 'E': 4, 'F': 5,
    'G': 6, 'H': 7, 'I': 8
}   # Center is always '4'
state = "---------"  # Initial empty board
turn = 0  # Keep track of moves
winStates = {
    'x': [
        r'x..x..x..', r'..x..x..x', r'...x...x...',
        r'x...x...x', r'.x...x...x', r'..x...x...x',
        r'x.x.x.x.', r'..x.x.x..'
    ],
    'o': [
        r'o..o..o..', r'..o..o..o', r'...o...o...',
        r'o...o...o', r'.o...o...o', r'..o...o...o',
        r'o.o.o.o.', r'..o.o.o..'
    ]
}
stateJournal = []  # Log of states and computer's moves: (state, move)
betaReward = 0.5  # Beta for reward
betaPunish = 0.5  # Beta for punishment
STM = {}

In [193]:
def assignSpaces(space):
    
    if space == 'A' or space == 'B': # orientation 1
        spaceMapping.update({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8})
        
    if space == 'C' or space == 'F': # orientation 2
        spaceMapping.update({'A': 6, 'B': 3, 'C': 0, 'D': 7, 'E': 4, 'F': 1, 'G': 8, 'H': 5, 'I': 2})
        
    if space == 'I' or space == 'H': # orientation 3
        spaceMapping.update({'A': 8, 'B': 7, 'C': 6, 'D': 5, 'E': 4, 'F': 3, 'G': 2, 'H': 1, 'I': 0})
        
    if space == 'G' or space == 'D': # orientation 4
        spaceMapping.update({'A': 2, 'B': 5, 'C': 8, 'D': 1, 'E': 4, 'F': 7, 'G': 0, 'H': 3, 'I': 6})

    

def showBoard():
    """
    Prints the board in a user-friendly format with position labels.
    """
    board = ""
    board += "  A | B | C\n"
    board += f"  {state[0]} | {state[1]} | {state[2]}\n"
    board += "------------- \n"
    board += "  D | E | F\n"
    board += f"  {state[3]} | {state[4]} | {state[5]}\n"
    board += "------------- \n"
    board += "  G | H | I\n"
    board += f"  {state[6]} | {state[7]} | {state[8]}\n"
    print(board)



def move(player):
    """
    Handles player and computer moves.
    Prompts the player for input, calls assignSpaces() when needed,
    and updates the game state.
    """
    global state, turn

    if player == 'o':  # Human player
        showBoard()
        while True:
            pos = input("Enter position (A-I): ").upper()
            if pos in spaceMapping and state[int(spaceMapping[pos])] == '-':
                state_list = list(state)
                state_list[int(spaceMapping[pos])] = player
                state = "".join(state_list)
                turn += 1
                break
            else:
                print("Invalid move. Try again.")
                
        #Assign spaces after first non-center move
        open_spaces = [s for s in 'ABCDFGH' if s in pos]
        if turn == 2 and len(open_spaces) > 0:
          assignSpaces(open_spaces[0])

    else:  # Computer 'x'
        
        pos = decide()
        print(f"Computer plays at: {list(spaceMapping.keys())[list(spaceMapping.values()).index(int(pos))]}")
        state_list = list(state)
        state_list[int(pos)] = player
        state = "".join(state_list)
        stateJournal.append((state, pos))  # Log the move
        turn += 1

        #Assign spaces after first non-center move
        open_spaces = [s for s in 'ABCDFGH' if s in pos]
        if turn == 1 and len(open_spaces) > 0:
          assignSpaces(open_spaces[0])

def decide():
    """
    The computer selects a move based on the STM probabilities.
    Handles the case where only one move is left.
    """
    global state, turn, STM

    empty_spaces = [i for i, char in enumerate(state) if char == '-']

    if len(empty_spaces) == 1:
        return str(empty_spaces[0])  # Last move

    
    current_stm = STM.get(turn, {}) #get STM for current turn, if not exist, return empty dict
    possible_moves = {}
    for space in empty_spaces:
        new_state = list(state)
        new_state[space] = 'x'
        possible_moves[str(space)] = "".join(new_state)

    
    
    next_moves_probabilities = current_stm.get(state, {}) #get next move probabilities for current state
    
    if not next_moves_probabilities: #if no probabilities, choose randomly
      move = random.choice(list(possible_moves.keys()))
      return move
    
    
    
    
    
    
    
    probabilities = []
    available_moves = []
    for move, next_state in possible_moves.items():
      prob = next_moves_probabilities.get(next_state, 0) #get probability for next state, if not exist, default to 0
      probabilities.append(prob)
      available_moves.append(move)
    
    
    
    
    
    
    # Probabilistic selection
    cumulative_probability = 0
    random_value = random.random()
    for i, probability in enumerate(probabilities):
        cumulative_probability += probability
        if random_value <= cumulative_probability:
            return available_moves[i]

    #Shouldn't happen, but return a random move if probabilities don't add up to 1
    return random.choice(available_moves)
    

def learn(result):
    """
    Updates the STM probabilities based on the game's outcome.
    Uses Algedonic compensation with betaReward and betaPunish.
    """
    global STM, stateJournal, betaReward, betaPunish

    for i, (encountered, decided) in enumerate(reversed(stateJournal)):
        if STM.get(len(stateJournal) - 1 - i) is None:
          STM[len(stateJournal) - 1 - i] = {}
        if STM.get(len(stateJournal) - 1 - i).get(encountered) is None:
          STM[len(stateJournal) - 1 - i][encountered] = {}

        next_moves = {}
        empty_spaces = [i for i, char in enumerate(encountered) if char == '-']
        for space in empty_spaces:
          new_state = list(encountered)
          new_state[space] = 'x'
          next_moves["".join(new_state)] = space
        
        for possibility in next_moves:
          if STM.get(len(stateJournal) - 1 - i).get(encountered).get(possibility) is None:
            STM[len(stateJournal) - 1 - i][encountered][possibility] = 0
        
        for possibility in next_moves:
            if possibility == stateJournal[len(stateJournal) - 1 - i][0]:
                if result == 'win' or result == 'tie':
                    beta = betaReward
                else:
                    beta = betaPunish
                
                old_probability = STM[len(stateJournal) - 1 - i][encountered][possibility]
                STM[len(stateJournal) - 1 - i][encountered][possibility] = old_probability + beta * (1 - old_probability)
            else:
                if result == 'win' or result == 'tie':
                    beta = betaReward
                else:
                    beta = betaPunish
                old_probability = STM[len(stateJournal) - 1 - i][encountered][possibility]
                STM[len(stateJournal) - 1 - i][encountered][possibility] = old_probability + beta * (0 - old_probability)
                
def checkWin(board):
    """
    Checks if the current board state is a win for 'x' or 'o'.
    Returns 'x', 'o', or None.
    """
    for player in ['x', 'o']:
        for pattern in winStates[player]:
            if re.match(pattern, board):
                return player
    return None

def branch(state, player):
  """
  Generates all possible next states from a given state.
  Used to build the STM.
  """
  next_states = []
  empty_spaces = [i for i, char in enumerate(state) if char == '-']
  for space in empty_spaces:
    new_state = list(state)
    new_state[space] = player
    next_states.append("".join(new_state))
  return next_states

In [194]:
def build_stm():
  """
  Builds the State Transition Matrix (STM) by exploring all possible game states.
  """
  global STM
  STM = {}
  q = ["---------"]  # Start with the initial empty board
  visited = set()
  turn_count = 0

  while q:
    current_state = q.pop(0)

    if current_state in visited:
      continue
    visited.add(current_state)
    
    #check if the current state is the last state
    if current_state.count('-') == 0:
      continue

    # Initialize STM level for the current turn if it doesn't exist
    if turn_count not in STM:
        STM[turn_count] = {}

    # Initialize STM entry for the current state if it doesn't exist
    if current_state not in STM[turn_count]:
        STM[turn_count][current_state] = {}
        
    next_states = []
    if turn_count % 2 == 0:
      next_states = branch(current_state, 'x')
    else:
      next_states = branch(current_state, 'o')
      
    for next_state in next_states:
      STM[turn_count][current_state][next_state] = 0
      q.append(next_state)
    
    turn_count += 1
  
  #normalize the probabilities
  for t in STM:
    for s in STM[t]:
      total_p = len(STM[t][s])
      for ns in STM[t][s]:
        STM[t][s][ns] = 1/total_p

In [195]:
betaReward = 0.5
betaPunish = 0.25

In [196]:
def playGame():
    """
    Facilitates a single game of Tic Tac Toe between the user and the program.
    The program learns continuously over multiple games.
    """
    global state, turn, stateJournal, spaceMapping, STM

    printStates = input("Print String states each turn? (y/n): ").lower().startswith('y') #

    state = "---------"
    turn = 0
    stateJournal = []
    spaceMapping = {'E': 4}  # Reset for a new game
    
    spaceMapping = {
        'A': 0, 'B': 1, 'C': 2,
        'D': 3, 'E': 4, 'F': 5,
        'G': 6, 'H': 7, 'I': 8
    }
    build_stm() #build the STM before the game starts

    while True:
        move('x')  # Computer's turn
        if printStates:
            print(f"State: {state}")
        winner = checkWin(state)
        if winner:
            showBoard()
            if winner == 'x':
                print("Computer wins! :)")
                learn('win')
            else:
                print("You win! :(")
                learn('lose')
            break
        if turn == 9:
            showBoard()
            print("It's a tie! :)")
            learn('tie')
            break

        move('o')  # Human's turn
        if printStates:
            print(f"State: {state}")
        winner = checkWin(state)
        if winner:
            showBoard()
            if winner == 'o':
                print("You win! :(")
                learn('lose')
            else:
                print("Computer wins! :)")
                learn('win')
            break
        if turn == 9:
            showBoard()
            print("It's a tie! :)")
            learn('tie')
            break



# Play the game!
playGame()

Computer plays at: A
State: x--------
  A | B | C
  x | - | -
------------- 
  D | E | F
  - | - | -
------------- 
  G | H | I
  - | - | -

State: x---o----
Computer plays at: H
State: x---o--x-
  A | B | C
  x | - | -
------------- 
  D | E | F
  - | o | -
------------- 
  G | H | I
  - | x | -

State: x--oo--x-
Computer plays at: G
State: x--oo-xx-
  A | B | C
  x | - | -
------------- 
  D | E | F
  o | o | -
------------- 
  G | H | I
  x | x | -

State: x--oo-xxo
Computer plays at: C
State: x-xoo-xxo
  A | B | C
  x | - | x
------------- 
  D | E | F
  o | o | -
------------- 
  G | H | I
  x | x | o

State: x-xoooxxo
Computer plays at: B
State: xxxoooxxo
  A | B | C
  x | x | x
------------- 
  D | E | F
  o | o | o
------------- 
  G | H | I
  x | x | o

It's a tie! :)


In [197]:
def print_stm(showForJustOneState=False, specificState="---------"):
    """
    Prints the State Transition Matrix (STM) for debugging purposes.
    Optionally, print only the transitions for a specific state.

    Args:
        showForJustOneState (bool, optional): If True, only print the transitions
            for the state specified by 'specificState'. Defaults to False.
        specificState (str, optional): The state to print transitions for, if
            'showForJustOneState' is True. Defaults to "---------".
    """
    global STM

    print("=====================================================")
    print("State Transition Matrix (STM)")
    print("=====================================================")

    for turn_num, transitions in STM.items():
        print(f"Turn {turn_num} ====================================")
        for start_state, end_states in transitions.items():
            if not showForJustOneState or start_state == specificState:
                print(f"{start_state}:")
                for end_state, probability in end_states.items():
                    print(f"    {end_state}: {probability}")

#print_stm() #uncomment to print STM